In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import gmaps
import os
import time
import datetime

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
# Creates lists for data to be stored 
date = []
place = []
mag = []
cdi = []
mmi = []
coordinates = []
lat = []
long = []

# 1906 San Francisco Earthquake Data
def earthquake(e):
    params = {"starttime": "1857-01-20", "endtime": "1906-04-20",
              "minmagnitude": "4.0", "maxradiuskm":600, "latitude": 34.21, "longitude":-118.57}
    data = requests.get(e, params = params)
    data = json.loads(data.text)
    return data

e = r"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
a = earthquake(e)

for i in (a["features"]):
    print(i["properties"]["time"], i["properties"]["place"],
    i["properties"]["cdi"], i["properties"]["mag"],i["properties"]["mmi"],i["geometry"]["coordinates"][0],i["geometry"]["coordinates"][1])

-2010353400000 8 km NE of Imperial, California None 6.28 None -115.5 32.9
-2010394053700 The 1906 San Francisco Earthquake None 7.9 8.997 -122.55 37.75
-2014342500000 5 km NE of Poway, California None 4.5 None -117 33
-2050738200000 21 km WSW of Bodfish, California None 5 None -118.7 35.5
-2064576840000 3 km SW of Winters, California None 4.5 None -122 38.5
-2095866660000 3 km ENE of Seven Trees, California None 5.8 None -121.8 37.3
-2100422880000 3 km S of Milpitas, California None 5.8 None -121.9 37.4
-2127652800000 5 km SSW of Los Alamos, California None 5.4 None -120.3 34.7
-2127920580000 7 km SSE of Orcutt, California None 5.4 None -120.4 34.8
-2131996500000 1 km WSW of Winchester, California None 4.5 None -117.1 33.7
-2133926940000 5 km S of Elmira, California None 5.4 None -121.9 38.3
-2172154500000 12 km NNW of Parkfield, California None 6.4 None -120.5 36
-2198625526000 4 km ENE of Aromas, California None 4.5 None -121.6 36.9
-2209548900000 Near San Jacinto, California None 6.

In [3]:
for i in (a["features"]):
    date.append(i["properties"]["time"])
    place.append(i["properties"]["place"])
    cdi.append(i["properties"]["cdi"])
    mag.append(i["properties"]["mag"])
    mmi.append(i["properties"]["mmi"])
    lat.append(i["geometry"]["coordinates"][0])
    long.append(i["geometry"]["coordinates"][1])
    
# create a dictionary for establishing dataframe
san_francisco = {
    "City":place,
    "Epoch Time":date,
    "Magnitude": mag,
    "MMI": mmi,
    "CDI":cdi,
    "Latitude":lat,
    "Longitude":long
}

san_fran_df = pd.DataFrame(san_francisco)
san_fran_df.head()

,City,Epoch Time,Magnitude,MMI,CDI,Latitude,Longitude
0,"8 km NE of Imperial, California",-2010353400000,6.28,NaN,None,-115.50,32.90
1,The 1906 San Francisco Earthquake,-2010394053700,7.90,8.997,None,-122.55,37.75
2,"5 km NE of Poway, California",-2014342500000,4.50,NaN,None,-117.00,33.00
3,"21 km WSW of Bodfish, California",-2050738200000,5.00,NaN,None,-118.70,35.50
4,"3 km SW of Winters, California",-2064576840000,4.50,NaN,None,-122.00,38.50


In [4]:
epoch_list = san_fran_df['Epoch Time'].tolist()
converted_list = []

for i in epoch_list:
    converted = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(i/1000.0))
    converted_list.append(converted)

In [5]:
san_fran_df['Date & Time'] = converted_list
san_fran_df.head()

,City,Epoch Time,Magnitude,MMI,CDI,Latitude,Longitude,Date & Time
0,"8 km NE of Imperial, California",-2010353400000,6.28,NaN,None,-115.50,32.90,1906-04-19 00:30:00
1,The 1906 San Francisco Earthquake,-2010394053700,7.90,8.997,None,-122.55,37.75,1906-04-18 13:12:26
2,"5 km NE of Poway, California",-2014342500000,4.50,NaN,None,-117.00,33.00,1906-03-03 20:25:00
3,"21 km WSW of Bodfish, California",-2050738200000,5.00,NaN,None,-118.70,35.50,1905-01-06 14:30:00
4,"3 km SW of Winters, California",-2064576840000,4.50,NaN,None,-122.00,38.50,1904-07-30 10:26:00


In [6]:
coordinates = san_fran_df[['Longitude', 'Latitude']]

In [7]:
# Plotting map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Adding marker layer
## TODO: Make markers interactive
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…